In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np
import pyreadstat

import requests
import time
import wget
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def sign_in_manager(driver : webdriver, email : str, password : str):
    # Go to the SynoInt login page
    driver.get("https://manager.synoint.com/login")
    
    # Enter the credentials and sign in
    driver.find_element(By.ID, "email").send_keys(email)
    driver.find_element(By.ID, "password").send_keys(password)
    driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()

Get the survey list assigned to the project

In [3]:
def get_survey_list(driver : webdriver, tracker_id : int):
    # Go to the project
    driver.get(f"https://manager.synoint.com/en/projects/{tracker_id}")

    # Wait until survey list loader is out
    element = WebDriverWait(driver, 10).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, "td.dataTables_empty")))

    # Read the table with the survey list
    survey_list = pd.read_html( driver.find_element(By.ID, "project-syno__surveys").get_attribute("outerHTML") )[0]
    
    # Return the dataframe with the survey list
    return survey_list

Generate a new raw data file (SPSS)

In [4]:
def generate_raw_data(driver : webdriver, survey_id : int):
    # Go to the results section
    driver.get(f"https://manager.synoint.com/en/syno-survey/surveys/{survey_id}/results")
    # Open the report modal
    driver.find_element(By.CSS_SELECTOR, ".file-export-form__button").click()
    driver.implicitly_wait(3)
    # File format: Raw data
    driver.find_element(By.CSS_SELECTOR, "label[for='export_form_fileFormat_1']").click()
    time.sleep(0.5)
    # Type: SPSS
    driver.find_element(By.CSS_SELECTOR, "label[for='export_form_fileType_3']").click()
    time.sleep(0.5)
    # Waiter until loads exportation options for SPSS
    element = WebDriverWait(driver, 10).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, ".file-export-preloader__holder")))
    # Completes only: Yes
    driver.find_element(By.CSS_SELECTOR, "label[for='export_form_completesOnly']").click()
    time.sleep(0.5)
    # Include IP: Yes
    driver.find_element(By.CSS_SELECTOR, "label[for='export_form_includeIp']").click()
    time.sleep(0.5)
    # Include panelist Id: Yes
    driver.find_element(By.CSS_SELECTOR, "label[for='export_form_includePanelistId']").click()
    time.sleep(0.5)
    # Include indentity Id: Yes
    driver.find_element(By.CSS_SELECTOR, "label[for='export_form_includeIdentityId']").click()
    time.sleep(2)
    # Click the export button
    driver.find_element(By.CSS_SELECTOR, ".file-export__submit-button").click()

    # Checks if element exists
    token_error = driver.find_elements(By.CSS_SELECTOR, ".export__form > div.pt-2 > span.alert")
    # If there is an error with the token form, resubmit the form again
    if len(token_error) > 0:
        print("Error token found!\t Resubmitting form again")
        driver.find_element(By.CSS_SELECTOR, ".file-export__submit-button").click()
    
    print(f"Report for survey {survey_id} is being generated")

Download the generated raw data (SPSS) in a local directory

In [5]:
def download_raw_data(driver : webdriver, survey_id : int):
     # Go to the generated reports section
    driver.get(f"https://manager.synoint.com/en/syno-survey/export/{survey_id}")
    # Waiter until loads exportation files list
    element = WebDriverWait(driver, 5).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, ".file-export-preloader__holder")))
    
    try:
        is_file_exported = WebDriverWait(driver, 5).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, "table > tbody > tr:nth-child(1) > td:nth-child(9) > i")))
        downloable_file = driver.find_element(By.CSS_SELECTOR, "table > tbody > tr:nth-child(1) > td:nth-child(9) > div").get_attribute("data-content")
        filename = wget.download(downloable_file, os.environ["DATA_DIR"])
        print(f"File {filename} was downloaded")
    except:
        print("Reports is still being generated, trying again...")
        download_raw_data(driver, survey_id) 

In [6]:
files = os.listdir(os.environ["DATA_DIR"])
for file in files:
    print(f"Removing file {os.environ['DATA_DIR'] + file}")
    os.remove(os.environ["DATA_DIR"] + file)

try:
    # Connect to the remote webdriver
    options = webdriver.EdgeOptions()
    options.headless = True
    # driver = webdriver.Edge("C:\Program Files\edgedriver_win64\msedgedriver.exe")
    driver = webdriver.Remote(command_executor = os.environ["REMOTE_SERVER"], options = options)

    sign_in_manager(driver, os.environ["SYNO_EMAIL"], os.environ["SYNO_PASSWORD"])
    survey_list = get_survey_list(driver, 4300)

    # Generate and download the raw data for all surveys in 4700
    for survey_id in survey_list["Id"]:
        print(f"Generating report for {survey_id}")
        generate_raw_data(driver, survey_id)

    for survey_id in survey_list["Id"]:
        print(f"Downloading raw data for {survey_id}")
        download_raw_data(driver, survey_id)
except Exception as e:
    print(f"Exception in webdriver: {e}")
finally:
    driver.quit()

Removing file data/Raw Data 337199 2024-02-24.sav
Removing file data/Raw Data 422493 2024-02-24.sav
Generating report for 665730
Report for survey 665730 is being generated
Generating report for 174031
Report for survey 174031 is being generated
Generating report for 156910
Report for survey 156910 is being generated
Generating report for 222861
Report for survey 222861 is being generated
Generating report for 818418
Report for survey 818418 is being generated
File data//Raw Data 665730 2024-02-24.sav was downloaded
File data//Raw Data 174031 2024-02-24.sav was downloaded
File data//Raw Data 156910 2024-02-24.sav was downloaded
File data//Raw Data 222861 2024-02-24.sav was downloaded
Reports is still being generated, trying again...
File data//Raw Data 818418 2024-02-24.sav was downloaded
